In [3]:
from local_tree_decomp import *

from local_helpers import *
from helper_functions import *

from local_hom_count import *
from local_hom_count_int import *
from local_hom_count_int_pre import *
from local_hom_count_int_dict import *


In [9]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)
    dict_int_rep_result = count_homomorphisms_int_dict(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result, dict_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print('result:', better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)

        print('\nDict int representation:')
        %timeit count_homomorphisms_int_dict(graph, target_graph)


In [12]:
graph = graphs.CompleteBipartiteGraph(2, 2)
target_graph = graphs.CompleteGraph(20)

speed_test(graph, target_graph, naive=False)


result: 130340

Basic:
617 ms ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Int representation:
344 ms ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Better int representation:
52.9 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

 Dict int representation:
83.8 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
